In [1]:
%%time
import pandas as pd

from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE

from time import strftime, strptime, mktime, time, ctime

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

import gc


/home/shj16110/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


CPU times: user 428 ms, sys: 623 ms, total: 1.05 s
Wall time: 10.2 s


In [2]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)

df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

s = StandardScaler()
df = s.fit_transform(df)
test = s.transform(test)

print df.shape, test.shape

(69713, 22)
(30037, 21)
(69713, 37416) (30037, 37416)
CPU times: user 58.6 s, sys: 36.6 s, total: 1min 35s
Wall time: 1min 35s


In [10]:
%%time
np.save('yesterday_train.npy', df)
np.save('yesterday_test.npy', test)

CPU times: user 54 ms, sys: 7.06 s, total: 7.11 s
Wall time: 11.5 s


In [13]:
!ls -sh|grep yes

8.4G yesterday_test.npy
 20G yesterday_train.npy


In [7]:
type(test)

numpy.ndarray

### from here

In [3]:
%%time
train = np.load('yesterday_train.npy')
test = np.load('yesterday_test.npy')
y = np.load('y.npy')
test_id = np.load('test_id.npy')

CPU times: user 0 ns, sys: 9.87 s, total: 9.87 s
Wall time: 15.9 s


In [5]:
%%time
m = lambda *x: [i.shape for i in x]
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state  = 7)
print m(X_train, y_train, X_test, y_test)

[(55770, 37416), (55770,), (13943, 37416), (13943,)]
CPU times: user 27.4 s, sys: 11 s, total: 38.4 s
Wall time: 38.4 s


In [6]:

%%time
d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

CPU times: user 21.8 s, sys: 9.26 s, total: 31.1 s
Wall time: 31.1 s


In [7]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.05,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':-1,
    'eval_metric':'auc'
    
}
watchlist = [(d_train, 'train'), (d_test, 'test')]



In [8]:
start=time()

clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=10, 
                verbose_eval=1,  maximize=True,
               )
                


'!!! used %.2f s'%(time()-start)

[0]	train-auc:0.755572	test-auc:0.74268
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 10 rounds.
[1]	train-auc:0.763693	test-auc:0.746725
[2]	train-auc:0.808163	test-auc:0.778945
[3]	train-auc:0.821418	test-auc:0.788371
[4]	train-auc:0.829662	test-auc:0.788434
[5]	train-auc:0.838111	test-auc:0.796159
[6]	train-auc:0.841357	test-auc:0.799753
[7]	train-auc:0.847234	test-auc:0.801516
[8]	train-auc:0.850079	test-auc:0.800837
[9]	train-auc:0.853872	test-auc:0.801561
[10]	train-auc:0.855823	test-auc:0.799704
[11]	train-auc:0.85742	test-auc:0.799223
[12]	train-auc:0.859456	test-auc:0.80132
[13]	train-auc:0.864468	test-auc:0.800919
[14]	train-auc:0.874764	test-auc:0.80946
[15]	train-auc:0.875935	test-auc:0.809061
[16]	train-auc:0.878595	test-auc:0.81055
[17]	train-auc:0.879211	test-auc:0.81049
[18]	train-auc:0.881494	test-auc:0.811516
[19]	train-auc:0.882554	test-auc:0.811061
[20]	train-auc:0.884943	test-auc:0.

KeyboardInterrupt: 

In [13]:
%%time
dtest = xgb.DMatrix(test)

CPU times: user 14.9 s, sys: 7.02 s, total: 21.9 s
Wall time: 21.9 s


In [14]:
%%time
predb = clf.predict(dtest)


CPU times: user 4.3 s, sys: 1 ms, total: 4.3 s
Wall time: 268 ms


___

In [19]:
submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(predb, columns=['Approved'])], axis=1)
submit.shape

(30037, 2)

In [21]:
submit.to_csv('xgb1.csv',index=False)

In [22]:
clf.save_model('xgb_0.852586.model')

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

In [3]:
clf

NameError: name 'clf' is not defined

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba